# Binary MNIST classification

In [1]:
import random
from engine import Value
import nn
import functional as F

# using torch to download MNIST dataset
import torch
import torchvision 
from torchvision import transforms

# MLP definition

In [2]:
class MLP(nn.Module):
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = []
        for i in range(len(nouts)):
            self.layers.append(nn.Linear(sz[i], sz[i+1]))
            self.layers.append(nn.Sigmoid())

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

# Training

In [3]:
train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=transforms.Compose([
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                           download=True)

In [4]:
# only use 0s and 1s
xs = []
ys = []

for x, y in train_dataset:
    if y == 0 or y == 1:
        xs.append(x)
        ys.append(y)

len(xs), len(ys)

In [6]:
# convert tensors to 1d python lists
xs = list(map(torch.flatten, xs))
xs = list(map(lambda x: x.tolist(), xs))

In [ ]:
model = MLP(784, [16, 16, 1])

In [15]:
# SGD: can change number of updates to tradeoff accuracy for time
for k in range(150):
  x = xs[k]
  y = ys[k]
  
  # forward pass
  ypred = model(x)[0]
  loss = F.binary_cross_entropy(ypred, y)
  
  # backward pass
  for p in model.parameters():
    p.grad = 0.0
  loss.backward()
  
  # update
  for p in model.parameters():
    p.data += -0.1 * p.grad
  
  if k % 10 == 0:
    print(k, loss.data)


0 1.9549271998637545
10 0.14510332591757447
20 2.1040245041199257
30 1.8790778677082107
40 0.2503963168376077
50 1.6017092910910464
60 1.0728002936430032
70 0.26511466522764543
80 0.3079954526857389
90 1.4351493885707065
100 0.22772366300577015
110 0.7792054677449994
120 0.9435080263722164
130 0.1265752438045917
140 0.9029145730399232


# Testing

In [16]:
test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False,
                                          transform=transforms.Compose([
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                          download=True)

In [17]:
# only use 0s and 1s
test_xs = []
test_ys = []

for x, y in test_dataset:
    if y == 0 or y == 1:
        test_xs.append(x)
        test_ys.append(y)

len(test_xs), len(test_ys)

(2115, 2115)

In [18]:
# convert tensors to 1d python lists
test_xs = list(map(torch.flatten, test_xs))
test_xs = list(map(lambda x: x.tolist(), test_xs))

In [19]:
correct = 0
total = 0
for x, y in zip(test_xs[:50], test_ys[:50]):
    pred = model(x)[0]
    num_pred = 1 if pred.data >= 0.5 else 0
    correct += (y == num_pred)
    total += 1

print(f"accuracy: {correct / total}")

accuracy: 0.9
